In [1]:
import numpy as np
import pandas as pd
test_features=["/Users/yangqingyi/Desktop/trainxgb/fuse_features.npy"]
train=np.load(test_features[0])
ytrain=np.squeeze(np.load("/Users/yangqingyi/Desktop/trainxgb/labels_target.npz")['arr_0'])

In [3]:
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error 
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
def xgb_evaluate(n_estimators,max_depth,subsample
                ,colsample_bytree,learning_rate):
    model=XGBRegressor(n_estimators=int(n_estimators),
                          max_depth=int(max_depth),
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          learning_rate=learning_rate,n_jobs=-1,
#                           tree_method='gpu_hist',
#                           predictor='gpu_predictor',
                          eval_metric="rmse")
    rmse_scores=[]
    kf = KFold(n_splits = 5)
    for train_ix, test_ix in tqdm(kf.split(train)):
        X_train, X_test = train[train_ix], train[test_ix]
        Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
        model.fit(X_train,Y_train)
        pred = model.predict(X_test)
        rmse_scores.append(-np.sqrt(mean_squared_error(Y_test,pred)))
    val=np.mean(rmse_scores)
    return val
pbounds = {'max_depth': (5,15),
           'n_estimators': (100,1000),
           "subsample":(0.001, 1),
           "colsample_bytree":(0.001,1),
           "learning_rate":(0.05,0.5)}
xgb_bo = BayesianOptimization(
        f=xgb_evaluate,   # Objective function
        pbounds=pbounds,  # Value space
        verbose=2,
        random_state=2,
)
xgb_bo.maximize(init_points=10,   # Steps of random search
                   n_iter=2,    # Number of iterations to perform Bayesian optimization
                   acq='ei')
print(xgb_bo.max)
res_xgb = xgb_bo.max
params_xgb = res_xgb['params']
for key in params_xgb:
    if key in ["max_depth","n_estimators"]:
        params_xgb[key]=int(params_xgb[key])

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------


5it [00:04,  1.10it/s]


| 1         | -8.921    | 0.4366    | 0.06167   | 10.5      | 491.8     | 0.4209    |


5it [00:01,  2.92it/s]


| 2         | -9.873    | 0.331     | 0.1421    | 11.19     | 369.7     | 0.2676    |


...

| 10        | -8.948    | 0.4281    | 0.2465    | 12.77     | 582.0     | 0.9538    |


5it [00:04,  1.18it/s]


| 11        | -8.73     | 0.3809    | 0.07164   | 6.116     | 568.7     | 0.3679    |


5it [00:00,  8.51it/s]

| 12        | -10.68    | 0.001     | 0.5       | 15.0      | 100.0     | 1.0       |
{'target': -8.729649822614714, 'params': {'colsample_bytree': 0.3808758277895758, 'learning_rate': 0.07163939963288986, 'max_depth': 6.116288851477059, 'n_estimators': 568.7468140644572, 'subsample': 0.367906040574913}}


In [4]:
from sklearn.ensemble import RandomForestRegressor
def rf_evaluate(n_estimators,max_depth,max_features
                ,min_samples_split):
    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                   max_depth=int(max_depth),
                                   max_features=min(max_features, 0.999),
                                   min_samples_split=int(min_samples_split),
                                   n_jobs=-1)
    kf=KFold(n_splits=5)
    rmse_scores=[]
    for train_ix, test_ix in tqdm(kf.split(train)):
        X_train, X_test = train[train_ix], train[test_ix]
        Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
        model.fit(X_train,Y_train)
        pred = model.predict(X_test)
        rmse_scores.append(-np.sqrt(mean_squared_error(Y_test,pred)))
    val=np.mean(rmse_scores)
    return val

pbounds = {'max_depth': (5, 15),
           'n_estimators': (100,1000),
           "max_features":(0.1, 0.999),
           "min_samples_split":(2,25)}
rf_bo = BayesianOptimization(
        f=rf_evaluate,   
        pbounds=pbounds,  
        verbose=2,  
        random_state=1,
)
rf_bo.maximize(init_points=5,   
                   n_iter=5,    
                   acq='ei')
print(rf_bo.max)
res_rf = rf_bo.max
params_rf = res_rf['params']
"""
Partial hyperparameter integerization
"""
for key in params_rf:
    if key in ["max_depth","min_samples_split","n_estimators"]:
        params_rf[key]=int(params_rf[key])

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------


5it [00:06,  1.34s/it]


| 1         | -8.491    | 9.17      | 0.7476    | 2.003     | 372.1     |


5it [00:02,  1.96it/s]


| 2         | -8.561    | 6.468     | 0.183     | 6.284     | 411.0     |


...

| 8         | -8.537    | 11.23     | 0.3007    | 24.06     | 824.2     |


5it [00:07,  1.59s/it]


| 9         | -8.481    | 5.212     | 0.8263    | 24.19     | 731.9     |


5it [00:05,  1.14s/it]

| 10        | -8.548    | 8.644     | 0.1201    | 5.506     | 974.4     |
{'target': -8.458459680412139, 'params': {'max_depth': 9.425573174554623, 'max_features': 0.47171016643272623, 'min_samples_split': 14.599513753070427, 'n_estimators': 769.7814234177125}}


In [6]:
from lightgbm import LGBMRegressor
def lgb_evaluate(n_estimators,max_depth,num_leaves
                ,subsample,colsample_bytree,learning_rate):
    model = LGBMRegressor(n_estimators=int(n_estimators),
                           max_depth=int(max_depth),
                           num_leaves=int(num_leaves),
                           subsample=subsample,
                           colsample_bytree=colsample_bytree,
                           learning_rate=learning_rate,n_jobs=-1)
    kf=KFold(n_splits=5)
    rmse_scores=[]
    for train_ix, test_ix in tqdm(kf.split(train)):
        X_train, X_test = train[train_ix], train[test_ix]
        Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
        model.fit(X_train,Y_train)
        pred = model.predict(X_test)
        rmse_scores.append(-np.sqrt(mean_squared_error(Y_test,pred)))
    val=np.mean(rmse_scores)
    return val
from random import choice
pbounds = {'max_depth': (3,7),
           "num_leaves":(5,31),
           'n_estimators': (100,1000),
           "subsample":(0.8, 1),
           "colsample_bytree":(0.8,1),
           "learning_rate":(0.0001,0.5)}
lgb_bo = BayesianOptimization(
        f=lgb_evaluate,   
        pbounds=pbounds,  
        verbose=2,  
        random_state=1,
)
lgb_bo.maximize(init_points=5,   
                   n_iter=5)
print(lgb_bo.max)
res_lgb = lgb_bo.max
params_lgb = res_lgb['params']
for key in params_lgb:
    if key in ["max_depth","num_leaves","n_estimators"]:
        params_lgb[key]=int(params_lgb[key])

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------


5it [00:01,  4.50it/s]


| 1         | -8.744    | 0.8834    | 0.3602    | 3.0       | 372.1     | 8.816     | 0.8185    |


5it [00:01,  2.81it/s]


| 2         | -8.844    | 0.8373    | 0.1728    | 4.587     | 584.9     | 15.9      | 0.937     |


...

| 8         | -8.804    | 0.9767    | 0.084     | 5.955     | 375.3     | 10.39     | 0.8588    |


5it [00:01,  3.93it/s]


| 9         | -9.542    | 0.8485    | 0.4725    | 3.615     | 374.6     | 9.987     | 0.8641    |


5it [00:01,  3.95it/s]

| 10        | -9.479    | 0.8298    | 0.3836    | 3.586     | 372.0     | 10.26     | 0.9356    |
{'target': -8.707259926928009, 'params': {'colsample_bytree': 0.8161109366683587, 'learning_rate': 0.11800247683159068, 'max_depth': 3.862318691254715, 'n_estimators': 371.92423104276736, 'num_leaves': 10.918975060678271, 'subsample': 0.9059866217469905}}


In [8]:
def model_fusion(a,b,c):
    xgb=XGBRegressor(**params_xgb,
                          eval_metric="rmse",n_jobs=-1)
    rf=RandomForestRegressor(**params_rf,n_jobs=-1)
    lgb=LGBMRegressor(**params_lgb,n_jobs=-1)
    kf=KFold(n_splits=5)
    rmse_scores=[]
    for train_ix, test_ix in tqdm(kf.split(train)):
        X_train, X_test = train[train_ix], train[test_ix]
        Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
        xgb.fit(X_train,Y_train)
        rf.fit(X_train,Y_train)
        lgb.fit(X_train,Y_train)
        pred = (a*xgb.predict(X_test)+b*rf.predict(X_test)+c*lgb.predict(X_test))/(a+b+c)
        rmse_scores.append(-np.sqrt(mean_squared_error(Y_test,pred)))
    val=np.mean(rmse_scores)
    return val
pbounds = {'a': (0.1,1),
           "b":(0.1,1),
           'c': (0.1,1)}
lgb_bo = BayesianOptimization(
        f=model_fusion,   
        pbounds=pbounds,  
        verbose=2,  
        random_state=1,
)
lgb_bo.maximize(init_points=5,   
                   n_iter=5)
print(lgb_bo.max)
res_lgb = lgb_bo.max
params_= res_lgb['params']

|   iter    |  target   |     a     |     b     |     c     |
-------------------------------------------------------------


5it [00:11,  2.38s/it]


| 1         | -8.462    | 0.4753    | 0.7483    | 0.1001    |


5it [00:12,  2.55s/it]


| 2         | -8.468    | 0.3721    | 0.2321    | 0.1831    |


...

| 8         | -8.431    | 0.3633    | 0.4918    | 0.4163    |


5it [00:11,  2.28s/it]


| 9         | -8.502    | 0.3048    | 0.24      | 0.9526    |


5it [00:14,  2.81s/it]

| 10        | -8.458    | 0.7536    | 0.7558    | 0.2752    |
{'target': -8.430807310328131, 'params': {'a': 0.3632867907176822, 'b': 0.491802199550132, 'c': 0.41631651088904625}}


In [9]:
xgb=XGBRegressor(**params_xgb,eval_metric="rmse",n_jobs=-1)
xgb.fit(train,ytrain)
rf=RandomForestRegressor(**params_rf,n_jobs=-1)
rf.fit(train,ytrain)
lgb=LGBMRegressor(**params_lgb,n_jobs=-1)
lgb.fit(train,ytrain)
pred = (params_["a"]*xgb.predict(train)+params_["b"]*rf.predict(train)+params_["c"]*lgb.predict(train))/(params_["a"]+\
                                                                                                     params_["b"]+\
                                                                                                     params_["c"])

In [10]:
import joblib
joblib.dump(xgb, "xgb.dat")

['xgb.dat']

In [11]:
joblib.dump(rf, "rf.pkl")

['rf.pkl']

In [12]:
joblib.dump(lgb, "lgb.pkl")

['lgb.pkl']